# dn-detr patched json

# anchor-detr & dn-detr

In [10]:
#!/usr/bin/env python3
import os
import json
from tqdm import tqdm

# ==================================================
# 配置
# ==================================================
VERSION = "0.5"
BASE_DIR = "/opt/data/private/BlackBox"
ANNOT_PATH = f"{BASE_DIR}/data/coco-patch-{VERSION}/annotations/instances_val2017.json"

# 模型结果路径列表
MODEL_RESULTS = [
    f"{BASE_DIR}/save-{VERSION}/attack/detection/anchor-detr/patch/res.json",
    f"{BASE_DIR}/save-{VERSION}/attack/detection/dn-detr/patch/res.json",
]

# ==================================================
# 辅助函数
# ==================================================
def load_image_and_patch_mapping(annotations_path):
    """
    从 patched annotations 文件中加载：
      - image_id → file_name
      - image_id → patch_regions（可能为 None）
    """
    with open(annotations_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    id_to_file = {}
    id_to_patch = {}
    for img in data.get("images", []):
        img_id = img["id"]
        id_to_file[img_id] = img.get("file_name", f"unknown_{img_id}.png")
        id_to_patch[img_id] = img.get("patch_regions", [])
    return id_to_file, id_to_patch


def convert_to_standard_format(input_path, image_mapping, patch_mapping, version):
    """将原始 JSON 转换为带 patch 信息的标准格式"""
    if not os.path.exists(input_path):
        print(f"⚠️ 文件不存在: {input_path}")
        return

    model_name = input_path.split("/")[-3]
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 支持两种结构：纯 list 或 dict{"annotations": [...]}
    detections = data.get("annotations", data) if isinstance(data, dict) else data

    results = []
    for det in detections:
        if det.get("category_id") != 1:
            continue

        image_id = int(det.get("image_id", -1))
        file_name = image_mapping.get(image_id, f"unknown_{image_id}.png")
        patch_regions = patch_mapping.get(image_id, [])

        bbox = det.get("bbox", [])
        if len(bbox) == 4:
            # 自动判断是否 xywh
            if bbox[2] < bbox[0] or bbox[3] < bbox[1]:
                x, y, w, h = bbox
                bbox = [x, y, x + w, y + h]

        score = float(det.get("score", 0.0))
        area = abs((bbox[2] - bbox[0]) * (bbox[3] - bbox[1]))

        results.append({
            "image_id": image_id,
            "file_name": file_name,
            "category_id": 1,
            "category_name": "person",
            "bbox": bbox,
            "score": score,
            "area": area,
            "patch_regions": patch_regions,  # ✅ 新增字段
            "model": model_name,
            "version": version
        })

    # 排序：image_id, category_id, score(desc)
    results = sorted(results, key=lambda x: (x["image_id"], x["category_id"], -x["score"]))

    # 保存结果
    output_path = os.path.join(os.path.dirname(input_path), "res-std.json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print(f"✅ {model_name}: {len(results)} 条记录 → {output_path}")

    # 打印前 30 行
    try:
        print("-" * 50)
        print(f"📄 {model_name} - {os.path.basename(output_path)} 前 30 行内容:")
        with open(output_path, "r", encoding="utf-8") as f:
            lines = f.readlines()
            for i, line in enumerate(lines[:30]):
                print(line.rstrip("\n"))
            if len(lines) > 30:
                print("... (已省略更多行)")
        print("-" * 50)
    except Exception as e:
        print(f"❌ 打印 JSON 内容失败: {e}")


# ==================================================
# 主程序
# ==================================================
def main():
    image_mapping, patch_mapping = load_image_and_patch_mapping(ANNOT_PATH)
    for path in tqdm(MODEL_RESULTS, desc="标准化转换中（含 patch）"):
        convert_to_standard_format(path, image_mapping, patch_mapping, VERSION)
    print("\n✅ 全部模型 patched JSON 已转换为标准格式（含 patch_regions）。")


if __name__ == "__main__":
    main()


标准化转换中（含 patch）: 100%|██████████| 2/2 [00:00<00:00,  3.34it/s]

✅ anchor-detr: 4865 条记录 → /opt/data/private/BlackBox/save-0.5/attack/detection/anchor-detr/patch/res-std.json
--------------------------------------------------
📄 anchor-detr - res-std.json 前 30 行内容:
[
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
    "bbox": [
      366.0715637207031,
      214.87013244628906,
      566.60888671875,
      711.7450714111328
    ],
    "score": 0.790438175201416,
    "area": 99641.9701248277,
    "patch_regions": [
      [
        457,
        430,
        522,
        495
      ]
    ],
    "model": "anchor-detr",
    "version": "0.5"
  },
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
... (已省略更多行)
--------------------------------------------------
✅ dn-detr: 1424 条记录 → /opt/data/private/BlackBox/save-0.5/attack/detection/dn-detr/patch/res-std.json
--------------------------------------------------
📄 dn-detr - res-std.json 前 

# sparse-detr

In [12]:
#!/usr/bin/env python3
import os
import json
from tqdm import tqdm

# =========================
# 配置
# =========================
VERSION = "0.5"
BASE_DIR = "/opt/data/private/BlackBox"
ANNOT_PATH = f"{BASE_DIR}/data/coco-patch-{VERSION}/annotations/instances_val2017.json"

# Sparse-DETR 原始结果（未标准化）路径
INPUT_PATH = f"{BASE_DIR}/save-{VERSION}/attack/detection/sparse-detr/patch/res.json"

# =========================
# 辅助函数
# =========================
def load_image_and_patch_mapping(annotations_path):
    """
    从 annotations 文件加载：
      - id_to_file: image_id → file_name
      - id_to_patch: image_id → patch_regions (list[list])
    """
    with open(annotations_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    id_to_file, id_to_patch = {}, {}
    for img in data.get("images", []):
        img_id = int(img["id"])
        id_to_file[img_id] = img.get("file_name", f"unknown_{img_id}.png")
        id_to_patch[img_id] = img.get("patch_regions", [])
    return id_to_file, id_to_patch


def coco_category_name(cat_id: int) -> str:
    """常见 COCO 类别名映射"""
    m = {
        1: "person", 2: "bicycle", 3: "car", 4: "motorcycle", 5: "airplane",
        6: "bus", 7: "train", 8: "truck", 9: "boat", 10: "traffic light",
        11: "fire hydrant", 13: "stop sign", 14: "parking meter", 15: "bench",
        16: "bird", 17: "cat", 18: "dog", 19: "horse", 20: "sheep", 21: "cow",
        22: "elephant", 23: "bear", 24: "zebra", 25: "giraffe",
        27: "backpack", 28: "umbrella", 31: "handbag", 32: "tie", 33: "suitcase",
    }
    return m.get(cat_id, f"cls_{cat_id}")


def xywh_to_xyxy(b):
    """将 [x, y, w, h] → [x1, y1, x2, y2]（Sparse-DETR 结果为 xywh）"""
    if not isinstance(b, (list, tuple)) or len(b) != 4:
        return b
    x, y, w, h = map(float, b)
    return [x, y, x + w, y + h]


def save_preview(model_name, output_path, max_lines=50):
    """打印前 max_lines 行便于快速校验"""
    try:
        print("-" * 50)
        print(f"📄 {model_name} - {os.path.basename(output_path)} 前 {max_lines} 行内容:")
        with open(output_path, "r", encoding="utf-8") as f:
            lines = f.readlines()
            for i, line in enumerate(lines[:max_lines]):
                print(line.rstrip("\n"))
            if len(lines) > max_lines:
                print("... (已省略更多行)")
        print("-" * 50)
    except Exception as e:
        print(f"❌ 打印 JSON 内容失败: {e}")


# =========================
# 主转换
# =========================
def main():
    print("=== Sparse-DETR (patched) → 标准化转换开始 ===")
    if not os.path.exists(INPUT_PATH):
        print(f"⚠️ 未找到输入文件: {INPUT_PATH}")
        return

    id_to_file, id_to_patch = load_image_and_patch_mapping(ANNOT_PATH)

    with open(INPUT_PATH, "r", encoding="utf-8") as f:
        data = json.load(f)

    detections = data if isinstance(data, list) else data.get("annotations", [])
    model_name = "sparse-detr"

    results = []
    for det in tqdm(detections, desc="标准化中"):
        try:
            image_id = int(det.get("image_id"))
        except Exception:
            continue

        file_name = id_to_file.get(image_id, f"unknown_{image_id}.png")
        patch_regions = id_to_patch.get(image_id, [])

        cat_id = int(det.get("category_id", -1))
        cat_name = coco_category_name(cat_id)
        bbox_xyxy = xywh_to_xyxy(det.get("bbox", []))
        score = float(det.get("score", 0.0))

        if not isinstance(bbox_xyxy, list) or len(bbox_xyxy) != 4:
            continue

        x1, y1, x2, y2 = bbox_xyxy
        area = abs((x2 - x1) * (y2 - y1))

        results.append({
            "image_id": image_id,
            "file_name": file_name,
            "category_id": cat_id,
            "category_name": cat_name,
            "bbox": [x1, y1, x2, y2],
            "score": score,
            "area": area,
            "patch_regions": patch_regions,  # ✅ 新增字段
            "model": model_name,
            "version": VERSION
        })

    # 按 image_id, category_id, -score 排序
    results = sorted(results, key=lambda x: (x["image_id"], x["category_id"], -x["score"]))

    # 输出
    output_path = os.path.join(os.path.dirname(INPUT_PATH), "res-std.json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print(f"✅ {model_name}: {len(results)} 条记录 → {output_path}")
    save_preview(model_name, output_path)
    print("=== 完成 ===")


if __name__ == "__main__":
    main()


=== Sparse-DETR (patched) → 标准化转换开始 ===


标准化中: 100%|██████████| 28800/28800 [00:00<00:00, 196836.10it/s]


✅ sparse-detr: 28800 条记录 → /opt/data/private/BlackBox/save-0.5/attack/detection/sparse-detr/patch/res-std.json
--------------------------------------------------
📄 sparse-detr - res-std.json 前 50 行内容:
[
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
    "bbox": [
      400.472,
      220.257,
      564.198,
      702.465
    ],
    "score": 0.89026,
    "area": 78949.98700800001,
    "patch_regions": [
      [
        457,
        430,
        522,
        495
      ]
    ],
    "model": "sparse-detr",
    "version": "0.5"
  },
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
    "bbox": [
      195.779,
      460.099,
      250.43599999999998,
      507.336
    ],
    "score": 0.47182,
    "area": 2581.8327090000002,
    "patch_regions": [
      [
        457,
        430,
        522,
        495
      ]
    ],
    "model": "sparse-detr",
    "version": "0.5"


# deformable-detr

In [13]:
#!/usr/bin/env python3
import os
import json
from tqdm import tqdm

# ==================================================
# 配置
# ==================================================
VERSION = "0.5"
BASE_DIR = "/opt/data/private/BlackBox"
ANNOT_PATH = f"{BASE_DIR}/data/coco-patch-{VERSION}/annotations/instances_val2017.json"

# Deformable-DETR 原始检测结果路径
INPUT_PATH = f"{BASE_DIR}/save-{VERSION}/attack/detection/deformable-detr/patch/res.json"

# ==================================================
# 辅助函数
# ==================================================
def load_image_and_patch_mapping(annotations_path):
    """
    从 annotations 文件加载:
      - id_to_file: image_id → file_name
      - id_to_patch: image_id → patch_regions (list[list])
    """
    with open(annotations_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    id_to_file, id_to_patch = {}, {}
    for img in data.get("images", []):
        img_id = int(img["id"])
        id_to_file[img_id] = img.get("file_name", f"unknown_{img_id}.png")
        id_to_patch[img_id] = img.get("patch_regions", [])
    return id_to_file, id_to_patch


def coco_category_name(cat_id: int) -> str:
    """COCO 类别名映射"""
    names = {
        1: "person", 2: "bicycle", 3: "car", 4: "motorcycle", 5: "airplane",
        6: "bus", 7: "train", 8: "truck", 9: "boat", 10: "traffic light",
        11: "fire hydrant", 13: "stop sign", 14: "parking meter", 15: "bench",
        16: "bird", 17: "cat", 18: "dog", 19: "horse", 20: "sheep", 21: "cow",
        22: "elephant", 23: "bear", 24: "zebra", 25: "giraffe",
        27: "backpack", 28: "umbrella", 31: "handbag", 32: "tie", 33: "suitcase",
    }
    return names.get(cat_id, f"cls_{cat_id}")


def save_preview(model_name, output_path, max_lines=30):
    """打印前 max_lines 行 JSON 内容"""
    try:
        print("-" * 50)
        print(f"📄 {model_name} - {os.path.basename(output_path)} 前 {max_lines} 行内容:")
        with open(output_path, "r", encoding="utf-8") as f:
            lines = f.readlines()
            for i, line in enumerate(lines[:max_lines]):
                print(line.rstrip("\n"))
            if len(lines) > max_lines:
                print("... (已省略更多行)")
        print("-" * 50)
    except Exception as e:
        print(f"❌ 打印 JSON 内容失败: {e}")

# ==================================================
# 主转换
# ==================================================
def main():
    print("=== Deformable-DETR (patched) → 标准化转换开始 ===")

    if not os.path.exists(INPUT_PATH):
        print(f"⚠️ 未找到输入文件: {INPUT_PATH}")
        return

    # 加载映射信息（包含 patch 信息）
    id_to_file, id_to_patch = load_image_and_patch_mapping(ANNOT_PATH)

    with open(INPUT_PATH, "r", encoding="utf-8") as f:
        data = json.load(f)

    detections = data if isinstance(data, list) else data.get("annotations", [])
    model_name = "deformable-detr"

    results = []
    for det in tqdm(detections, desc="标准化中"):
        try:
            image_id = int(det.get("image_id"))
        except Exception:
            continue

        file_name = id_to_file.get(image_id, f"unknown_{image_id}.png")
        patch_regions = id_to_patch.get(image_id, [])

        cat_id = int(det.get("category_id", -1))
        cat_name = coco_category_name(cat_id)

        bbox = det.get("bbox", [])
        if not bbox or len(bbox) != 4:
            continue

        # bbox 已是 [x1, y1, x2, y2]
        x1, y1, x2, y2 = map(float, bbox)
        if x2 <= x1 or y2 <= y1:
            continue  # 跳过异常框

        score = float(det.get("score", 0.0))
        area = abs((x2 - x1) * (y2 - y1))

        results.append({
            "image_id": image_id,
            "file_name": file_name,
            "category_id": cat_id,
            "category_name": cat_name,
            "bbox": [x1, y1, x2, y2],
            "score": score,
            "area": area,
            "patch_regions": patch_regions,  # ✅ 新增字段
            "model": model_name,
            "version": VERSION
        })

    # ✅ 排序
    results = sorted(results, key=lambda x: (x["image_id"], x["category_id"], -x["score"]))

    # ✅ 保存
    output_path = os.path.join(os.path.dirname(INPUT_PATH), "res-std.json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print(f"✅ {model_name}: {len(results)} 条记录 → {output_path}")
    save_preview(model_name, output_path)
    print("=== 转换完成 ===")


if __name__ == "__main__":
    main()


=== Deformable-DETR (patched) → 标准化转换开始 ===


标准化中: 100%|██████████| 28800/28800 [00:00<00:00, 305918.65it/s]


✅ deformable-detr: 28800 条记录 → /opt/data/private/BlackBox/save-0.5/attack/detection/deformable-detr/patch/res-std.json
--------------------------------------------------
📄 deformable-detr - res-std.json 前 30 行内容:
[
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
    "bbox": [
      403.53240966796875,
      221.9440460205078,
      577.9172973632812,
      699.607421875
    ],
    "score": 0.86912602186203,
    "area": 83297.27415454946,
    "patch_regions": [
      [
        457,
        430,
        522,
        495
      ]
    ],
    "model": "deformable-detr",
    "version": "0.5"
  },
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
... (已省略更多行)
--------------------------------------------------
=== 转换完成 ===


# detr

In [14]:
#!/usr/bin/env python3
import os
import json
from tqdm import tqdm

# ==================================================
# 配置
# ==================================================
VERSION = "0.5"
BASE_DIR = "/opt/data/private/BlackBox"
ANNOT_PATH = f"{BASE_DIR}/data/coco-patch-{VERSION}/annotations/instances_val2017.json"
INPUT_PATH = f"{BASE_DIR}/save-{VERSION}/attack/detection/detr/patch/res.json"

# ==================================================
# 辅助函数
# ==================================================
def load_image_and_patch_mapping(annotations_path):
    """
    从 annotations 文件加载:
      - id_to_file: image_id → file_name
      - id_to_patch: image_id → patch_regions (list[list])
    """
    with open(annotations_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    id_to_file, id_to_patch = {}, {}
    for img in data.get("images", []):
        img_id = int(img["id"])
        id_to_file[img_id] = img.get("file_name", f"unknown_{img_id}.png")
        id_to_patch[img_id] = img.get("patch_regions", [])
    return id_to_file, id_to_patch


def coco_category_name(cat_id: int) -> str:
    """COCO 类别名映射"""
    names = {
        1: "person", 2: "bicycle", 3: "car", 4: "motorcycle", 5: "airplane",
        6: "bus", 7: "train", 8: "truck", 9: "boat", 10: "traffic light",
        11: "fire hydrant", 13: "stop sign", 14: "parking meter", 15: "bench",
        16: "bird", 17: "cat", 18: "dog", 19: "horse", 20: "sheep", 21: "cow",
        22: "elephant", 23: "bear", 24: "zebra", 25: "giraffe",
        27: "backpack", 28: "umbrella", 31: "handbag", 32: "tie", 33: "suitcase",
    }
    return names.get(cat_id, f"cls_{cat_id}")


def save_preview(model_name, output_path, max_lines=30):
    """打印前 max_lines 行 JSON 内容"""
    try:
        print("-" * 50)
        print(f"📄 {model_name} - {os.path.basename(output_path)} 前 {max_lines} 行内容:")
        with open(output_path, "r", encoding="utf-8") as f:
            lines = f.readlines()
            for i, line in enumerate(lines[:max_lines]):
                print(line.rstrip("\n"))
            if len(lines) > max_lines:
                print("... (已省略更多行)")
        print("-" * 50)
    except Exception as e:
        print(f"❌ 打印 JSON 内容失败: {e}")

# ==================================================
# 主逻辑
# ==================================================
def main():
    print("=== DETR (patched) → 标准化转换开始 ===")

    if not os.path.exists(INPUT_PATH):
        print(f"⚠️ 未找到输入文件: {INPUT_PATH}")
        return

    # 加载 file_name 与 patch_regions 映射
    id_to_file, id_to_patch = load_image_and_patch_mapping(ANNOT_PATH)

    # 加载检测结果
    with open(INPUT_PATH, "r", encoding="utf-8") as f:
        data = json.load(f)
    detections = data if isinstance(data, list) else data.get("annotations", [])

    model_name = "detr"
    results = []

    for det in tqdm(detections, desc="标准化中"):
        try:
            image_id = int(det.get("image_id", -1))
        except Exception:
            continue

        file_name = id_to_file.get(image_id, f"unknown_{image_id}.png")
        patch_regions = id_to_patch.get(image_id, [])

        cat_id = int(det.get("category_id", -1))
        cat_name = coco_category_name(cat_id)

        bbox = det.get("bbox", [])
        if not bbox or len(bbox) != 4:
            continue

        # DETR 的 bbox 已经是 [x1, y1, x2, y2]
        x1, y1, x2, y2 = map(float, bbox)
        if x2 <= x1 or y2 <= y1:
            continue  # 跳过异常框

        score = float(det.get("score", 0.0))
        area = abs((x2 - x1) * (y2 - y1))

        results.append({
            "image_id": image_id,
            "file_name": file_name,
            "category_id": cat_id,
            "category_name": cat_name,
            "bbox": [x1, y1, x2, y2],
            "score": score,
            "area": area,
            "patch_regions": patch_regions,  # ✅ 新增字段
            "model": model_name,
            "version": VERSION
        })

    # ✅ 排序：image_id 升序，类别升序，score 降序
    results = sorted(results, key=lambda x: (x["image_id"], x["category_id"], -x["score"]))

    # ✅ 输出保存
    output_path = os.path.join(os.path.dirname(INPUT_PATH), "res-std.json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print(f"✅ {model_name}: {len(results)} 条记录 → {output_path}")
    save_preview(model_name, output_path)
    print("=== 转换完成 ===")

if __name__ == "__main__":
    main()


=== DETR (patched) → 标准化转换开始 ===


标准化中: 100%|██████████| 28800/28800 [00:00<00:00, 242126.53it/s]


✅ detr: 28800 条记录 → /opt/data/private/BlackBox/save-0.5/attack/detection/detr/patch/res-std.json
--------------------------------------------------
📄 detr - res-std.json 前 30 行内容:
[
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
    "bbox": [
      314.3294677734375,
      221.06639099121094,
      569.1385498046875,
      702.9391479492188
    ],
    "score": 0.9997256398200989,
    "area": 122785.5548563376,
    "patch_regions": [
      [
        457,
        430,
        522,
        495
      ]
    ],
    "model": "detr",
    "version": "0.5"
  },
  {
    "image_id": 1,
    "file_name": "crop001545.png",
    "category_id": 1,
    "category_name": "person",
... (已省略更多行)
--------------------------------------------------
=== 转换完成 ===
